## Senior Design Team 5
# Project Gaea

Welcome to our analysis code. In here, we will populate the map using the Kepler.Gl interface, which will plot the most current heat adn average temperature data in the state of California, along with retailer locations where residents under the wildfire threat could access PPE.

In [81]:
# Importing libraries.
import pandas as pd
from keplergl import KeplerGl
import geopandas as gpd
import requests
from pandas.io.json import json_normalize
import sys
import codecs

if sys.version_info[0] >= 3:
    unicode = str

### Step 1: Creating the Map using Kepler

In [82]:
# Load an empty map
# Specify the properties of the map here.

map_1 = KeplerGl(height=1000)
map_1

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(height=1000)

In [136]:
# Quick access - save it to HTML
map_1.save_to_html(file_name = 'gaeaDemo.html')

Map saved to gaeaDemo.html!


### Step 2: Load in heat data, API-based or raw. Start parsing and filtering.

In this step, first we will handle data parsing with the .csv file format and in the latter stages, these will be quesry-obtained JSON files.

### National Interagency Fire Center, active fire incident data

In [83]:
# Getting data from query URL.
ifa_url = 'https://opendata.arcgis.com/datasets/68637d248eb24d0d853342cba02d4af7_0.geojson'
ifa_data = requests.get(ifa_url)

In [84]:
# Present the response, if it is 200 it means success.
ifa_data

<Response [200]>

In [85]:
# Extract data in JSON format.
ifa_json = ifa_data.json()
ifa_json

{'type': 'FeatureCollection',
 'name': 'Current_Wildfire_Points',
 'crs': {'type': 'name',
  'properties': {'name': 'urn:ogc:def:crs:OGC:1.3:CRS84'}},
 'features': [{'type': 'Feature',
   'properties': {'OBJECTID': 119266,
    'ABCDMisc': None,
    'ADSPermissionState': 'DEFAULT',
    'CalculatedAcres': None,
    'ContainmentDateTime': '1970/01/01 00:00:00+00',
    'ControlDateTime': '1970/01/01 00:00:00+00',
    'CreatedOnDateTime': 1559316251757,
    'DailyAcres': None,
    'DiscoveryAcres': None,
    'DispatchCenterID': 'ORRVC',
    'FinalFireReportApprovedByTitle': None,
    'FinalFireReportApprovedByUnit': None,
    'FinalFireReportApprovedDate': '1970/01/01 00:00:00+00',
    'FireBehaviorGeneral': None,
    'FireBehaviorGeneral1': None,
    'FireBehaviorGeneral2': None,
    'FireBehaviorGeneral3': None,
    'FireCause': 'Unknown',
    'FireCauseGeneral': None,
    'FireCauseSpecific': None,
    'FireCode': None,
    'FireDepartmentID': None,
    'FireDiscoveryDateTime': '2019/05/

In [86]:
map_1.add_data(data=ifa_json,name='Current Wildfires')

### Wildfire Perimeter Data - Open Data California

In [ ]:
# Loading in data from an API.
url = 'https://opendata.arcgis.com/datasets/f72ebe741e3b4f0db376b4e765728339_0.geojson'
r = requests.get(url)

In [ ]:
# If response is 200, means it communicated. 
r

In [ ]:
# Extract data in JSON format.
json = r.json()
json

### Active Wildfire Data - active, last updated August, 2020

In [ ]:
active_data = pd.read_json('https://query.data.world/s/er4lejrygunujrb6fdqfisezawntuc')
active_data.head()

In [ ]:
active_data.columns

In [ ]:
map_1.add_data(data=active_data,name='Active Wildfires')

### NASA MODIS Heat Data - 2019

In [ ]:
df=pd.read_csv('modis_2019_United_States.csv')
df.head()

In [ ]:
df.columns

In [ ]:
# Add the data to the map.
map_1.add_data(data=df,name='Heatmap of NASA MODIS 2019')

### Firefighter Recorded Incidents at California, 2019

In [ ]:
df1=pd.read_csv('California_Fire_Incidents.csv')
df1.head()

In [ ]:
# Add the data to the map.
map_1.add_data(data=df1,name='Firefighter Incident Data')

## Step 3 - Get Target stock data from API

In this step, we query the Target.com servers based on the provided zipcode of California and the radius parameter - to get the product  availability.

### First, we find the Target store locations based on current location and ZIP.

In [ ]:
target_url = "https://target-com-store-product-reviews-locations-data.p.rapidapi.com/location/search"

In [ ]:
## Modify this zipcode based on area.

target_querystring = {"zip":"93702","radius":"100"}

## This API key is obtained by signing up for the API. 

target_headers = {
    'x-rapidapi-key': "8d73142587msh490ff325621437ep18babajsn6aa45b41eda4",
    'x-rapidapi-host': "target-com-store-product-reviews-locations-data.p.rapidapi.com"
    }

In [ ]:
target_response = requests.request("GET", target_url, headers=target_headers, params=target_querystring)

target_location_json = target_response.json()
type(target_location_json['locations'][0])

In [ ]:
# Parsing this geojson

target_df = json_normalize(target_location_json['locations'])
target_df_1 = pd.DataFrame(target_df)
#map_2.add_data(data=target_df_1,name='target try')

#locs = 'properties.geometry.coordinates'

In [ ]:
# Transform this to a dataframe
#  = pd.DataFrame(target_location_json['locations'])
# target_df.head()

In [ ]:
# Add the data to the map.
map_2.add_data(data=target_df,name='Target Retailers')

In [ ]:
#df2=pd.read_csv('targets.csv')
#df2.head()

This data is extracted from our local server, in a .csv format and shows the current Target department store locations. Updated recently on 11/02/2020.

In [104]:
# Read in csv for Target locations. 
t = pd.read_csv('calitarget.csv', encoding='latin-1')

In [105]:
# Sanity check
t.head()

,X.locale,Address.AddressLine1,Address.AddressLine2,Address.City,Address.CountryName,Address.County,Address.FormattedAddress,Address.IntersectionDescription,Address.Latitude,Address.Longitude,...,Is24Hours.Sat,IsOpen.Sat,Summary.Sat,ThruTime.Sat,BeginTime.Sun,Is24Hours.Sun,IsOpen.Sun,Summary.Sun,ThruTime.Sun,AllCapability
0,en-US,400 Oxford Exchange Blvd,NaN,Oxford,United States,Calhoun,"400 Oxford Exchange Blvd, Oxford, AL 36203-3459",SEC I-20 & Golden Creek Rd,33.608825,-85.783182,...,False,True,8:00 a.m.-11:00 p.m.,23:00:00,8:00:00,False,True,8:00 a.m.-10:00 p.m.,22:00:00,"['CVS pharmacy', 'Café', 'Mobile Kiosk', 'Star..."
1,en-US,2640 Enterprise Dr,NaN,Opelika,United States,Lee,"2640 Enterprise Dr, Opelika, AL 36801-1511",NWC I-85 & Hwy 280,32.618108,-85.410636,...,False,True,8:00 a.m.-11:00 p.m.,23:00:00,8:00:00,False,True,8:00 a.m.-10:00 p.m.,22:00:00,"['CVS pharmacy', 'Cafe-Pizza', 'Fresh Grocery'..."
2,en-US,1129 S Park St,NaN,Carrollton,United States,Carroll,"1129 S Park St, Carrollton, GA 30117-4429",NWC US 27 & Hwy 166 Bypass,33.558920,-85.076473,...,False,True,8:00 a.m.-10:00 p.m.,22:00:00,8:00:00,False,True,8:00 a.m.-9:00 p.m.,21:00:00,NaN
3,en-US,555 Bullsboro Dr,NaN,Newnan,United States,Coweta,"555 Bullsboro Dr, Newnan, GA 30265-1045",SWC I-85 & Rte 34,33.392907,-84.758875,...,False,True,8:00 a.m.-11:00 p.m.,23:00:00,8:00:00,False,True,8:00 a.m.-10:00 p.m.,22:00:00,"['CVS pharmacy', 'Café', 'Fresh Grocery', 'Mob..."
4,en-US,1591 Bradley Park Dr,NaN,Columbus,United States,Muscogee,"1591 Bradley Park Dr, Columbus, GA 31904-3071",NEC Bradley Pk & JR Allen Pkwy,32.534649,-84.971073,...,False,True,8:00 a.m.-11:00 p.m.,23:00:00,8:00:00,False,True,8:00 a.m.-10:00 p.m.,22:00:00,"['CVS pharmacy', 'Café', 'Mobile Kiosk', 'Phot..."


In [106]:
# Add data to map
map_1.add_data(data=t,name='Target Retailers')

### Mid-step: converting the map to HTML page for easy access

In [135]:
map_1.save_to_html(file_name = 'gaeaDemo.html')

Map saved to gaeaDemo.html!


### We also include data on all California drugstores.

In [118]:
# Read in csv for CA Pharmacies' locations. 
p = pd.read_csv('refilter.csv', encoding='latin-1')

In [119]:
#Processing and splitting the coordinates
p[['x', 'y']] = p['CalcLocation'].str.split(',', 1, expand=True)
p.head()

,CalcLocation,Name,Address,City,State,Zip,formated_phone,Status,Icon,Type,x,y
0,"34.05224200,-118.47358800",PHARMACA INTEGRATIVE PHARMACY,12025 SAN VICENTE BLVD,LOS ANGELES,CA,900494999,310-476-1237,open,small_green,Pharmacy,34.05224200,-118.47358800
1,"34.44166500,-119.26200400",OJAI REXALL DRUGS,1125 MARICOPA HWY,OJAI,CA,930233196,805-646-4361,open,small_green,Pharmacy,34.44166500,-119.26200400
2,"36.56798000,-119.61165500",FOX DRUG STORE,1939 HIGH ST,SELMA,CA,93662,559-896-1645,open,small_green,Pharmacy,36.56798000,-119.61165500
3,"36.98205300,-122.00743100",SAFEWAY PHARMACY #0799,117 MORRISSEY BLVD,SANTA CRUZ,CA,950621540,831-426-8911,open,small_green,Pharmacy,36.98205300,-122.00743100
4,"37.96247100,-121.33772300",SAFEWAY PHARMACY #1769,2808 COUNTRY CLUB BLVD,STOCKTON,CA,952043957,209-461-5560,open,small_green,Pharmacy,37.96247100,-121.33772300


In [120]:
# Add pharmacy data to map
map_1.add_data(data=p,name='Pharmacies')

### As an additional retail option, we include Walmart locations.

In [133]:
# Read in csv for CA Walmart locations. 
w = pd.read_csv('walmartCA.csv', encoding='latin-1')
# Sanity check
w.head()

,name,url,street_address,city,state,zip_code,country,phone_number_1,phone_number_2,fax_1,...,email_2,website,open_hours,latitude,longitude,facebook,twitter,instagram,pinterest,youtube
0,Stockton Supercenter,https://www.walmart.com/store/1554/stockton-ca...,3223 E Hammer Ln,Stockton,CA,95212,US,209-473-2796,NaN,NaN,...,NaN,NaN,"monday - friday : 06:00-24:00, saturday : 06:0...",38.023504,-121.273567,NaN,NaN,NaN,NaN,NaN
1,El Centro Supercenter,https://www.walmart.com/store/1555/el-centro-c...,2150 N Waterman Ave,El Centro,CA,92243,US,760-337-1600,NaN,NaN,...,NaN,NaN,"monday - friday : 00:00-24:00, saturday : 00:0...",32.813411,-115.577324,NaN,NaN,NaN,NaN,NaN
2,Lancaster Supercenter,https://www.walmart.com/store/1563/lancaster-c...,44665 Valley Central Way,Lancaster,CA,93536,US,661-940-8744,NaN,NaN,...,NaN,NaN,"monday - friday : 06:00-24:00, saturday : 06:0...",34.693755,-118.174112,NaN,NaN,NaN,NaN,NaN
3,Oroville Supercenter,https://www.walmart.com/store/1575/oroville-ca...,465 Cal Oak Rd,Oroville,CA,95965,US,530-534-1082,NaN,NaN,...,NaN,NaN,"monday - friday : 06:00-24:00, saturday : 06:0...",39.496607,-121.573430,NaN,NaN,NaN,NaN,NaN
4,Madera Store,https://www.walmart.com/store/1583/madera-ca/d...,1977 W Cleveland Ave,Madera,CA,93637,US,559-675-9212,NaN,NaN,...,NaN,NaN,"monday - friday : 07:00-23:00, saturday : 07:0...",36.975625,-120.080610,NaN,NaN,NaN,NaN,NaN


In [134]:
# Add Walmart location data to map
map_1.add_data(data=w,name='Walmarts')

## Step 4 - Scraping Product Price Data from Retailers

Test run - open a session with the Target retailer, check the LOCAL availability / price of a product given the ID/TCIN.

In [121]:
# Open a new session with the retailer. 
s = requests.session()
s.get('https://www.target.com')

<Response [200]>

In [122]:
# Try to login with visitor id
key = s.cookies['visitorId']
location = s.cookies['GuestLocation'].split('|')[0]

In [123]:
# Queries the stores NEARBY - ONLY DOES SO FOR LOCAL STORES. 
store_id = requests.get('https://redsky.target.com/v3/stores/nearby/%s?key=%s&limit=1&within=100&unit=mile' %(location, key)).json()
store_id = store_id[0]['locations'][0]['location_id']

In [128]:
# Takes in the product ID that we are looking for. 
product_id = '79991725'  # This is a disposable facemask.
url1 = 'https://redsky.target.com/web/pdp_location/v1/tcin/%s' %product_id
payload = {
'pricing_store_id': store_id,
'key': key}


In [129]:
jsonData = requests.get(url1, params=payload).json()
df_test = pd.DataFrame(jsonData['price'], index=[0])
df_test.head()

,tcin,location_id,reg_retail,current_retail,current_retail_start_timestamp,current_retail_end_timestamp,default_price,formatted_current_price,formatted_current_price_type,is_current_price_range
0,79991725,3227,14.69,14.69,2020-11-12T16:18:19Z,9999-12-31T00:00:00Z,False,$14.69,reg,False
